In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import os
import numpy as np
import pandas as pd

In [3]:
import lib
import plotting

In [4]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read("config.cfg")

['config.cfg']

In [5]:
from tqdm._tqdm_notebook import tqdm_notebook as tqdm # we manually import the notebook submodule as the normal one struggles with jupyterlab
tqdm.pandas() # this enables us to use progress_apply instead of apply

In [11]:
results_prefix = cfg.get("results", "prefix")

sequence_file = os.path.join(cfg.get("directory", "exchange"), results_prefix + "001_clustering.p")
labels_file = os.path.join(cfg.get("directory", "exchange"), "[wikidata]004_labels_readable.p")

In [12]:
df = pd.read_pickle(sequence_file)
df.head()

,bot_name,bot_sequence,length,length_nobreak,sequence,sequence_readable,user_id,user_name,stat_dist,kmeans_4,kmeans_8
0,False,False,1517,1385,"[REVERT_MANUAL, BREAK, CLAIM_CREATE, BREAK, CL...","[Revert Item, Break, Create Claim, Break, Crea...",1,Hoo man,"[0.033592832488441825, 0.029154644609674315, 0...",0,3
1,False,False,44,23,"[SITELINK_ADD, BREAK, SITELINK_ADD, BREAK, SIT...","[Add Sitelink, Break, Add Sitelink, Break, Add...",100012,Kane 14,"[0.02586206896551723, 0.025862068965517227, 0....",2,1
2,False,False,17,12,"[LABEL_UPDATE, ALIAS_ADD, BREAK, CLAIM_CREATE,...","[Edit Label, Add Item Alias, Break, Create Cla...",1000197,Zaizone,"[0.026205469193709592, 0.07779210211701826, 0....",1,5
3,False,False,91,82,"[SITELINK_UPDATE, BREAK, LABEL_ADD, CLAIM_CREA...","[Edit Sitelink, Break, Add Label, Create Claim...",1001159,Gogo hr,"[0.04118566424353361, 0.03162035177772598, 0.0...",0,0
4,False,False,21,12,"[DESCRIPTION_ADD, BREAK, DESCRIPTION_UPDATE, D...","[Add Description, Break, Edit Description, Add...",1001194,Phylosofo,"[0.08620689655172406, 0.025862068965517255, 0....",1,7


In [13]:
labels = pd.read_pickle(labels_file)['label'].values

In [14]:
num_clusters = 4
df_selector = "kmeans_{n}".format(n=num_clusters)

In [15]:
cluster_group = df.groupby(df_selector)

In [19]:
results = []
for i, g in cluster_group:
    actions = g["sequence"].apply(len)
    actions = g["length_nobreak"]
    res = {
        "cluster": i,
        "population": len(g),
        "actions_sum": actions.sum(),
        "actions_mean": actions.mean(),
        "actions_median": actions.median(),
        "actions_max": actions.max(),
        "actions_min": actions.min()
    }
    
    #for i, r in g.iterrows():
    #    if len(r['sequence']) == actions.max():
    #        print(r["user_id"])
    results.append(res)
    
df_res = pd.DataFrame(results)
df_res

,actions_max,actions_mean,actions_median,actions_min,actions_sum,cluster,population
0,1098458,4821.861259,46.0,10,48030560,0,9961
1,1574515,1870.649995,31.0,10,20085169,1,10737
2,86583,319.509446,41.0,10,3974378,2,12439
3,16405,151.873961,31.0,10,803717,3,5292
